In [ ]:
#WEEK 5

In [ ]:
#"""problem 1"""
#find the 5 most profitable products

#pandas solution
import pandas as pd

online_orders = online_orders[online_orders["date"].dt.month <=6]
online_orders["total"] = online_orders["cost_in_dollars"] * online_orders["units_sold"]
online_orders.groupby("product_id")["total"].sum().nlargest(5)


#SQL solution
select product_id, sum(units_sold*cost_in_dollars) as total from online_orders
group by product_id
order by total DESC
limit 5;

In [ ]:
#"""problem 2"""
#Find the 3 most profitable companies in the entire world. Output the result along with the corresponding company name.
#Sort the result based on profits in descending order.

#pandas solution
import pandas as pd

df = forbes_global_2010_2014[["company","profits"]].nlargest(3,"profits")


#SQL solution
select company, profits from forbes_global_2010_2014
order by profits DESC
limit 3;

In [ ]:
#"""problem 3"""
#Calculate each user's average session time.

#pandas solution
import pandas as pd

facebook_web_log['day'] = facebook_web_log['timestamp'].dt.date
load = facebook_web_log[facebook_web_log["action"] == "page_load"]
exit = facebook_web_log[facebook_web_log["action"] == "page_exit"]

load = load.groupby(['user_id','day'],as_index = False).max()
exit = exit.groupby(['user_id','day'],as_index = False).min()

result = pd.merge(load,exit,how="left",on=["user_id","day"])
result = result[result["timestamp_x"] < result["timestamp_y"]]

result['duration'] = (result['timestamp_y'] - result['timestamp_x']).dt.total_seconds()
result.groupby('user_id',as_index=False).mean()



#SQL solution
with min_max as (select user_id,date(timestamp),
max(case 
    when action = 'page_load' then timestamp
    end) as pg_load,
min(case 
    when action = 'page_exit' then timestamp
    end)  as pg_exit
from facebook_web_log group by 1,2)

select user_id,avg(pg_exit-pg_load) as avg_session_time
from min_max
group by 1 having avg(pg_exit-pg_load) is not null ;